In [1]:
!pip install torch tensorflow transformers flask rdkit
!pip install -U bitsandbytes
!pip install -U accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = "/content/drive/MyDrive/fine_tuned_gpt2"
tokenizer_GPT = GPT2Tokenizer.from_pretrained(model_path, local_files_only=True)
model_GPT = GPT2LMHeadModel.from_pretrained(model_path, local_files_only=True)

print("model GPT2 loaded successfully")

model loaded successfully


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_lora_path = "/content/drive/MyDrive/smiles-lora"
model_llama = AutoModelForCausalLM.from_pretrained(model_lora_path)
tokenizer_llama = AutoTokenizer.from_pretrained(model_lora_path)
model_llama = model_llama.to("cuda")

print("model llama3 loaded successfully")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

model llama3 loaded successfully


In [6]:
import re
import torch
import time
from rdkit import Chem

def clean_smiles(generated):
    return re.sub(r'[^A-Za-z0-9@+\-\[\]\(\)=#$%.]', '', generated)

def is_valid_smiles(smiles):
    try:
        return Chem.MolFromSmiles(smiles) is not None
    except:
        return False

def generate_until_valid_smiles_GPT(seed_smiles, model, tokenizer, max_length=10, timeout_seconds=120):
    model.eval()
    device = next(model.parameters()).device
    input_ids = tokenizer.encode(seed_smiles, return_tensors='pt').to(device)

    min_length = len(seed_smiles) + 1
    assert min_length <= max_length, "Seed SMILES is too long for the specified max length!"

    start_time = time.time()

    while True:
        for curr_length in range(min_length, max_length + 1):
            elapsed_time = time.time() - start_time
            if elapsed_time > timeout_seconds:
                print(f"⏱️ Timeout reached ({int(elapsed_time)}s). Returning original SMILES: {seed_smiles}")
                return seed_smiles

            output = model.generate(
                input_ids=input_ids,
                max_length=curr_length,
                num_return_sequences=1,
                do_sample=True,
                top_k=50,
                top_p=0.95,
                temperature=0.7
            )

            decoded = tokenizer.decode(output[0], skip_special_tokens=True)
            cleaned = clean_smiles(decoded)

            if is_valid_smiles(cleaned):
                print(f"✅ Valid SMILES (length={curr_length}): {cleaned}")
                return cleaned
            else:
                print(f"❌ Invalid (length={curr_length}): {cleaned}")

In [11]:
def generate_until_valid_smiles_llama(seed_smiles, model, tokenizer, max_new_tokens=20, timeout_seconds=120):
    model.eval()
    device = next(model.parameters()).device
    inputs = tokenizer(seed_smiles, return_tensors="pt").to(device)

    start_time = time.time()

    while True:
        elapsed_time = time.time() - start_time
        if elapsed_time > timeout_seconds:
            print(f"⏱️ Timeout reached ({int(elapsed_time)}s). Returning original seed Smile: {seed_smiles}")
            return seed_smiles

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                top_p=0.95,
                temperature=0.7,
                repetition_penalty=1.1,
                num_return_sequences=1  # generate only 1 SMILES
            )

        raw_smiles = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned = clean_smiles(raw_smiles)

        if is_valid_smiles(cleaned):
            print(f"✅ Valid SMILES: {cleaned}")
            return cleaned
        else:
            print(f"❌ Invalid: {cleaned}")

In [12]:
seed_smiles = "CCO"
generated_smiles = generate_until_valid_smiles_llama(seed_smiles, model_llama, tokenizer_llama)
print("Generated valid SMILES:", generated_smiles)

[08:39:56] SMILES Parse Error: syntax error while parsing: CCOC(=O)c1ccccc1eXxm[B9G
[08:39:56] SMILES Parse Error: check for mistakes around position 17:
[08:39:56] CCOC(=O)c1ccccc1eXxm[B9G
[08:39:56] ~~~~~~~~~~~~~~~~^
[08:39:56] SMILES Parse Error: Failed parsing SMILES 'CCOC(=O)c1ccccc1eXxm[B9G' for input: 'CCOC(=O)c1ccccc1eXxm[B9G'


❌ Invalid: CCOC(=O)c1ccccc1eXxm[B9G
✅ Valid SMILES: CCOC(=O)c1ccccc1
Generated valid SMILES: CCOC(=O)c1ccccc1


In [13]:
!pip install pyngrok

In [14]:
import flask
from pyngrok import ngrok
from rdkit import Chem
from rdkit.Chem import QED
import threading

ngrok.set_auth_token("2w0ZwN5aAw5PyD0KEeJcuNxxUMx_3uJXr2hWsTowi7bSokxNW")

app = flask.Flask(__name__)

@app.route('/fineTunedGPT2', methods=['POST'])
def generate_smile_GPT2():
  data = flask.request.get_json()
  seed_smile = data['seed']
  valid_smile = generate_until_valid_smiles_GPT(
      seed_smiles=seed_smile,  # Your seed molecule
      model=model_GPT,
      tokenizer=tokenizer_GPT,
      max_length=10
  )
  mol = Chem.MolFromSmiles(valid_smile)
  qed_score = 0
  if mol:
    qed_score = QED.qed(mol)

  return flask.jsonify({
      'valid_smile': valid_smile,
      'qed_score': qed_score
  })

@app.route('/llama3', methods=['POST'])
def generate_smile_LLAMA3():
  data = flask.request.get_json()
  seed_smile = data['seed']
  valid_smile = generate_until_valid_smiles_llama(
      seed_smiles=seed_smile,  # Your seed molecule
      model=model_llama,
      tokenizer=tokenizer_llama,
      max_new_tokens=20
  )
  mol = Chem.MolFromSmiles(valid_smile)
  qed_score = 0
  if mol:
    qed_score = QED.qed(mol)

  return flask.jsonify({
      'valid_smile': valid_smile,
      'qed_score': qed_score
  })

def run_flask():
    app.run(port=5000)

thread = threading.Thread(target=run_flask)
thread.start()

# Set up ngrok tunnel
public_url = ngrok.connect(5000)
print("🚀 Your Flask app is live at:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


🚀 Your Flask app is live at: NgrokTunnel: "https://79f9-35-240-222-110.ngrok-free.app" -> "http://localhost:5000"
